In [5]:
import requests
import json
from datetime import datetime
from typing import Dict, List, Optional

In [6]:
tokens_s1 = '/tokens_obtenidos/tokens.json'
credenciales = '../EndPointsAPIS/EndPoints/endpointsS1.json'

#### Funciones

In [7]:
class APIClient:
    def __init__(self, suppliers_config: List[Dict], tokens_config: Dict):
        """
        Inicializa el cliente API con la configuración de proveedores y tokens
        
        Args:
            suppliers_config: Lista de configuraciones de proveedores
            tokens_config: Diccionario con los tokens de acceso por proveedor
        """
        self.suppliers = {s["supplier_id"]: s for s in suppliers_config}
        self.tokens = tokens_config

    def _get_token(self, supplier_id: str) -> Optional[str]:
        """Obtiene el token válido para un proveedor específico"""
        if supplier_id not in self.tokens:
            return None
            
        token_info = self.tokens[supplier_id]
        token_timestamp = datetime.fromisoformat(token_info["timestamp"])
        current_time = datetime.now()
        
        # Verificar si el token ha expirado
        time_diff = (current_time - token_timestamp).total_seconds()
        if time_diff >= token_info["expires_in"]:
            print(f"Token expirado para {supplier_id}")
            return None
            
        return token_info["access_token"]

    def query_endpoint(self, supplier_id: str) -> Dict:
        """
        Consulta el endpoint para un proveedor específico
        
        Args:
            supplier_id: Identificador del proveedor
            
        Returns:
            Dict con la respuesta del endpoint o mensaje de error
        """
        if supplier_id not in self.suppliers:
            return {"error": f"Proveedor no encontrado: {supplier_id}"}
            
        supplier = self.suppliers[supplier_id]
        token = self._get_token(supplier_id)
        
        if not token:
            return {"error": f"Token no válido para: {supplier_id}"}
            
        headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/json"
        }
        
        try:
            response = requests.get(supplier["url"], headers=headers, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return {"error": f"Error al consultar endpoint: {str(e)}"}

    def query_all_endpoints(self) -> Dict[str, Dict]:
        """
        Consulta todos los endpoints disponibles
        
        Returns:
            Dict con las respuestas de todos los endpoints
        """
        results = {}
        for supplier_id in self.suppliers:
            results[supplier_id] = self.query_endpoint(supplier_id)
        return results

# Ejemplo de uso:
def main():
    # Cargar configuraciones desde archivos
    with open(credenciales, "r") as f:
        suppliers_config = json.load(f)
    
    with open(tokens_s1, "r") as f:
        tokens_config = json.load(f)
    
    # Crear instancia del cliente
    client = APIClient(suppliers_config, tokens_config)
    
    # Consultar un endpoint específico
    result = client.query_endpoint("SESEA_AGUASCALIENTES")
    print(f"Resultado para SESEA_AGUASCALIENTES: {result}")
    
    # O consultar todos los endpoints
    all_results = client.query_all_endpoints()
    print("\nResultados de todos los endpoints:")
    for supplier_id, result in all_results.items():
        print(f"{supplier_id}: {result}")

if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: '../EndPointsAPIS/EndPoints/endpointsS1.json'